In [1]:
import tensorflow as tf

In [3]:
# Load and preprocess images
import tensorflow as tf
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'frames',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'frames',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 13 files belonging to 2 classes.
Using 11 files for training.
Found 13 files belonging to 2 classes.
Using 2 files for validation.


In [5]:
# Build and compile the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Save the model
model.save('jumping_jack_model.h5')


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5455 - loss: 0.6887 - val_accuracy: 0.5000 - val_loss: 0.5907
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9091 - loss: 0.3837 - val_accuracy: 0.5000 - val_loss: 4.6229
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4545 - loss: 5.2328 - val_accuracy: 0.5000 - val_loss: 0.5818
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4545 - loss: 0.6676 - val_accuracy: 0.5000 - val_loss: 0.9769
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5455 - loss: 0.9020 - val_accuracy: 0.5000 - val_loss: 0.8509
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5455 - loss: 0.7970 - val_accuracy: 0.5000 - val_loss: 0.5512
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6364 - loss: 0.5613 - val_accuracy: 0.5000 - val_loss: 0.5893
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4545 - loss: 0.6720 - val_accuracy: 0.5000 - val_loss: 0.4647
Epoch 9/10
1/1

In [ ]:
import cv2
import tensorflow as tf
import mediapipe as mp
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('jumping_jack_model.h5')

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define the dimensions of the output images
image_width = 224
image_height = 224

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Convert the grayscale image to RGB
    image_rgb = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)

    # Process the frame and extract pose landmarks
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Create a blank image
        blank_image = np.zeros((image_height, image_width, 3), dtype=np.uint8)

        # Extract relevant landmarks (e.g., for arms and legs)
        landmarks = results.pose_landmarks.landmark

        # Loop through the landmarks and draw them on the blank image
        for lm in landmarks:
            x = int(lm.x * image_width)
            y = int(lm.y * image_height)
            cv2.circle(blank_image, (x, y), 1, (255, 255, 255), -1)

        # Resize the image to match the model's input dimensions
        image_resized = cv2.resize(blank_image, (image_width, image_height))

        # Convert the image to the format expected by the model
        image_input = image_resized.reshape((1, image_width, image_height, 3))

        # Predict using the trained model
        prediction = model.predict(image_input)

        # Determine if the jump is correct or incorrect
        if prediction > 0.5:
            label = 'Correct'
        else:
            label = 'Incorrect'

        # Display the label on the frame
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Jumping Jack Evaluation', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\ahmed\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 789ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 